In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
%matplotlib inline
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats

In [ ]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
with open('df_all_cols.pkl', 'rb') as file:
    df = pkl.load(file)

# Linear Regression (sklearn)

In [ ]:
# features that I wanted to keep
df_features_cols = ['num_ratings', 'num_reviews', 'made_it', 'servings',
       'calories', 'num_photos', 'oven_temp', 'prep_time_minutes', 'egg_whole',
       'vanilla_extract_ml', 'salt_g', 'cinnamon_g', 'baking_soda_g',
       'baking_powder_g', 'water_ml', 'cocoa_powder_g', 'fats',
       'sugars', 'flours', 'nuts', 'chocolate_chips',
       'num_ingredients', 'fats_flours_ratio', 'fats_sugars_ratio',
       'sugars_flours_ratio', 'chips_nuts_ratio', 'white_brown_ratio']

In [ ]:
# manual train-test-holdout split
X, X_test, y, y_test = train_test_split(df[df_features_cols], df['rating'], test_size=0.2, random_state=59)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=59)

In [ ]:
m = LinearRegression()
m.fit(X_train, y_train)
print(m.score(X_train, y_train), m.score(X_val, y_val), m.score(X_test, y_test))

In [ ]:
# create a new DataFrame with predictions and residuals
y_pred = m.predict(X_test)
resid_df = pd.DataFrame(data={'y_pred': y_pred, 'residual': y_test - y_pred})

In [ ]:
# Plot residuals
plt.figure(figsize=(12,8))
plt.scatter(y_pred, resid_df['residual'])
plt.title('Residuals of Linear Regression on Rating', fontsize=30)
plt.ylabel('Residual', fontsize=30)
plt.xlabel('Predicted Rating', fontsize=30)
plt.tick_params('both', labelsize=30)
plt.tight_layout()
plt.savefig('lr_residuals.png',transparent=True)

In [ ]:
# SAVE MODEL
with open('lr_model.pkl', 'wb') as file:
    pkl.dump(m, file)

# Linear Regression (StatsModels)

In [ ]:
# same as above except statsmodels gives more statistics
m_sm = sm.OLS(y_train, sm.add_constant(X_train)).fit()
m_sm.summary()

In [ ]:
# SAVE MODEL
with open('lr_sm_model.pkl', 'wb') as file:
    pkl.dump(m_sm, file)

# Polynomial Features 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
pf = PolynomialFeatures(degree=2)

In [ ]:
poly_df = pd.DataFrame(pf.fit_transform(df[df_features_cols]), columns=pf.get_feature_names(input_features=df_features_cols))

In [ ]:
# manual train-val-holdout split  
X2, X2_test, y2, y2_test = train_test_split(poly_df, df['rating'], test_size=0.2, random_state=59)
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.25, random_state=59)

# Linear Regression with Polynomial Features

In [ ]:
m2 = LinearRegression()
m2.fit(X2_train, y2_train)
print(m2.score(X2_train, y2_train), m2.score(X2_val, y2_val), m2.score(X2_test, y2_test))
print(list(sorted(zip(pf.get_feature_names(input_features=df_features_cols), m2.coef_), key=lambda x:abs(x[1]), reverse=True))[:10])

In [ ]:
# SAVE MODEL
with open('lr_pf_model.pkl', 'wb') as file:
    pkl.dump(m2, file)

# LASSO Regression with Polynomial Features

In [ ]:
# scale features before using regularization
std = StandardScaler()  
X2_train_std = std.fit_transform(X2_train)  
X2_val_std = std.transform(X2_val)
X2_test_std = std.transform(X2_test)

In [ ]:
# grid search of alphas 
alphalist = 10**(np.linspace(-5,5,1000))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

In [ ]:
for i, alpha in enumerate(alphalist):
    lasso = Lasso(alpha=alpha)
    lasso.fit(X2_train_std, y2_train)
    err_vec_val[i] = lasso.score(X2_test_std, y2_test)
    
    ### other metrics to evaluate alpha
    #val_set_pred = lasso.predict(X_val_std)
    #err_vec_val[i] = metrics.mean_absolute_error(y_val, val_set_pred)
    #err_vec_val[i] = np.log(metrics.mean_squared_error(y_val, val_set_pred))

In [ ]:
best_alpha = alphalist[np.argmax(err_vec_val)]

In [ ]:
plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
m_lasso = Lasso(alpha=best_alpha)
m_lasso.fit(X2_train_std, y2_train)
print(m_lasso.score(X2_train_std, y2_train), m_lasso.score(X2_val_std, y2_val), m_lasso.score(X2_test_std, y2_test))

In [ ]:
# list important features 
print(sorted(list(zip(pf.get_feature_names(input_features=df_features_cols), m_lasso.coef_)), 
             key=lambda x:abs(x[1]), reverse=True))

In [ ]:
print(mean_absolute_error(y2_test, m_lasso.predict(X2_test_std)))
print(mean_squared_error(y2_test, m_lasso.predict(X2_test_std)))
print(np.sqrt(mean_squared_error(y2_test, m_lasso.predict(X2_test_std))))

In [ ]:
# SAVE MODEL
with open('lasso_pf_model.pkl', 'wb') as file:
    pkl.dump(m_lasso, file)

# Ridge Regression with Polynomial Features

In [ ]:
# scale features before using regularization
std = StandardScaler()  
X2_train_std = std.fit_transform(X2_train)  
X2_val_std = std.transform(X2_val)
X2_test_std = std.transform(X2_test)

In [ ]:
# grid search of alphas 
alphalist = 10**(np.linspace(-5,5,1000))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

In [ ]:
for i, alpha in enumerate(alphalist):
    ridge = Ridge(alpha=alpha)
    ridge.fit(X2_train_std, y2_train)
    err_vec_val[i] = ridge.score(X2_test_std, y2_test)
    
    ### other metrics to evaluate alpha
    #val_set_pred = lasso.predict(X_val_std)
    #err_vec_val[i] = metrics.mean_absolute_error(y_val, val_set_pred)
    #err_vec_val[i] = np.log(metrics.mean_squared_error(y_val, val_set_pred))

In [ ]:
best_alpha = alphalist[np.argmax(err_vec_val)]

In [ ]:
plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
m_ridge = Ridge(alpha=best_alpha)
m_ridge.fit(X2_train_std, y_train)
print(m_ridge.score(X2_train_std, y2_train), m_ridge.score(X2_val_std, y2_val), m_ridge.score(X2_test_std, y2_test))

In [ ]:
# list important features 
print(sorted(list(zip(pf.get_feature_names(input_features=df_features_cols), m_ridge.coef_)), 
             key=lambda x:abs(x[1]), reverse=True))

In [ ]:
print(mean_absolute_error(y2_test, m_ridge.predict(X2_test_std)))
print(mean_squared_error(y2_test, m_ridge.predict(X2_test_std)))
print(np.sqrt(mean_squared_error(y2_test, m_ridge.predict(X2_test_std))))

In [ ]:
# SAVE MODEL
with open('ridge_pf_model.pkl', 'wb') as file:
    pkl.dump(m_ridge, file)

# Random Forest 

In [ ]:
rfr = RandomForestRegressor(n_estimators=30, min_samples_split=20, random_state=32)  
rfr.fit(X_train, y_train)  
y_pred = rfr.predict(X_test)

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred))) 

In [ ]:
print(rfr.score(X_train, y_train), rfr.score(X_val, y_val), rfr.score(X_test, y_test))

In [ ]:
list(sorted((zip(df_features_cols, rfr.feature_importances_)), key=lambda x: abs(x[1]), reverse=True))

In [ ]:
plt.figure(figsize=(12,20))
plt.subplot(2,1,1)
plt.scatter(resid_df['y_pred'], resid_df['residual'])
plt.title('Residuals of Linear Regression', fontsize=30)
plt.ylabel('Residual', fontsize=30)
plt.xlabel('Predicted Rating', fontsize=30)
plt.tick_params('both', labelsize=30)
plt.subplot(2,1,2)
plt.scatter(rfr.predict(X_test), y_test - rfr.predict(X_test))
plt.title('Residuals of Random Forest Regression', fontsize=30)
plt.ylabel('Residual', fontsize=30)
plt.xlabel('Predicted Rating', fontsize=30)
plt.tick_params('both', labelsize=30)
plt.tight_layout()
plt.savefig('rf_residuals.png',transparent=True)

In [ ]:
# SAVE MODEL
with open('rf_model.pkl', 'wb') as file:
    pkl.dump(rfr, file)